In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [22]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [4]:
df_fake.head()


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [23]:
df_true.head(5)


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [24]:
df_fake["class"] = 0
df_true["class"] = 1

In [25]:
df_fake.shape

(23481, 5)

In [26]:
df_merge = pd.concat([df_fake, df_true], axis=0)

In [27]:
df = df_merge.drop(["title", "subject", "date"], axis=1)

In [28]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [29]:
df = df.sample(frac=1).reset_index(drop=True)

In [30]:
df.head(10)

,text,class
0,BEIJING (Reuters) - China said on Thursday it ...,1
1,Hillary Clinton: If I talked about droning Jul...,0
2,BEIJING (Reuters) - Venezuela s close ally Chi...,1
3,"MINOT AIR FORCE BASE, N.D. (Reuters) - In this...",1
4,A Texas federal judge appointed by Obama has a...,0
5,,0
6,Putin Tells Russian Army To Prepare For WWWIII...,0
7,Because NYC Firefighters don t have enough to ...,0
8,It is becoming increasingly difficult to separ...,0
9,SEOUL (Reuters) - When North Korean leader Kim...,1


In [31]:
#cleaning up the text
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [44]:
df.head(10)

,text,class
0,beijing reuters china said on thursday it ...,1
1,hillary clinton if i talked about droning jul...,0
2,beijing reuters venezuela s close ally chi...,1
3,minot air force base n d reuters in this...,1
4,a texas federal judge appointed by obama has a...,0
5,,0
6,putin tells russian army to prepare for wwwiii...,0
7,because nyc firefighters don t have enough to ...,0
8,it is becoming increasingly difficult to separ...,0
9,seoul reuters when north korean leader kim...,1


In [46]:
#find the integer in the dataset
x.head()

0    beijing  reuters    china said on thursday it ...
1    hillary clinton  if i talked about droning jul...
2    beijing  reuters    venezuela s close ally chi...
3    minot air force base  n d   reuters    in this...
4    a texas federal judge appointed by obama has a...
Name: text, dtype: object

In [33]:
df["text"] = df["text"].apply(wordopt)

In [35]:
x = df["text"]
y = df["class"]

In [37]:
#Splitting Training and Testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [18]:
#convert to string
x_train = x_train.astype(str)
y_train = y_train.astype(str)
x_test = x_test.astype(str)
y_test = y_test.astype(str)


In [47]:
#convert to vector  
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer(lowercase=False)
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [48]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(xv_train, y_train)
lr_pred = lr.predict(xv_test)
print("Accuracy of Logistic Regression: ", accuracy_score(y_test, lr_pred))

Accuracy of Logistic Regression:  0.9871714922048997


In [49]:
print(classification_report(y_test, lr_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5841
           1       0.98      0.99      0.99      5384

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [50]:
#Model Testing
def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "True News"
    

def predict_news(news):
    news = [news]
    vect_news = vectorization.transform(news).toarray()
    my_prediction = lr.predict(vect_news)
    return output_label(my_prediction[0])


In [51]:
news = input("Enter the news: ")
predict_news(news)

'True News'

In [52]:
#saving the model
import pickle
pickle.dump(lr, open('model.pkl','wb'))